In [1]:
import torch
torch.cuda.is_available()

/opt/conda/envs/llm-python3-11/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
from transformers import LlamaForCausalLM, CodeLlamaTokenizer

/opt/conda/envs/llm-python3-11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

In [4]:
# This runs on GPU
# The GPU is likely to run out of memory
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf").to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.87s/it]


In [5]:
# This runs on cpu
model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf")

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.03s/it]


In [8]:
PROMPT = '''def remove_non_ascii(s: str) -> str:
""" <FILL_ME>
return result
'''

# This will force a run on GPU
# input_ids = tokenizer(PROMPT, return_tensors="pt").to(device)["input_ids"]  # Ensure tokenizer runs on GPU

input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"] 
generated_ids = model.generate(input_ids, max_new_tokens=128)
filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]

print(PROMPT.replace("<FILL_ME>", filling))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def remove_non_ascii(s: str) -> str:
""" Remove non-ASCII characters from a string.

Args:
    s (str): The string to remove non-ASCII characters from.

Returns:
    str: The string with non-ASCII characters removed.
"""

result = ""
for c in s:
    if ord(c) < 128:
        result += c
return result



In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()